<a href="https://colab.research.google.com/github/twaldburger/flood475-presenter/blob/master/geo475_flood_modelling_in_gee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import ee
import geemap
import geemap.colormaps as cm
import time

## set some parameters, please update if required
PROJECT_ID = 'ee-timwaldburger-flood475' # your GEE project id
SAMPLE_SIZE = 100 # number of training locations per flood event and class
SEED = 342014 # for reproducible results

## connect to GEE
try:
    ee.Initialize()
except ee.EEException:
    ee.Authenticate()
    ee.Initialize(project=PROJECT_ID)

---
# Data exploration and visualization

In [54]:
## define the datasets we want to use for our model
globalFlood = ee.ImageCollection("GLOBAL_FLOOD_DB/MODIS_EVENTS/V1")
dem = ee.ImageCollection('COPERNICUS/DEM/GLO30') \
        .select('DEM')
landcover = ee.ImageCollection('ESA/WorldCover/v200')
hydro = ee.Image('MERIT/Hydro/v1_0_1')
prec = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY') \
         .select('total_precipitation')
runoffPotential = ee.Image('projects/sat-io/open-datasets/HiHydroSoilv2_0/Hydrologic_Soil_Group_250m') \
                    .remap([1, 2, 3, 4, 14, 24, 34], [1, 2, 3, 4, 1, 3, 4])

> **Task:** Get an overview on the datasets we use. What data do they provide? Who produced them? What is their spatial and temporal resolution? A good starting point is the [Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets).  
**@presenter:** Ask for people to explain the datasets. Ask them to mention special characteristics (e.g. very low resolution) and explain where they see problems that should be kept in mind.

In [3]:
## subtract the permanent water bodies from the flooded areas
def subtractPermanentWater(img):
  flood = img.select('flooded')
  perm = img.select('jrc_perm_water')
  return flood.multiply(perm.eq(0))
flood = globalFlood.map(subtractPermanentWater).sum()
# flood_global_max = flood.reduceRegion(geometry=globalFlood.geometry(), reducer=ee.Reducer.max()).getInfo()['flooded']

## color flooded areas in blue
flood_vis = {'min':0, 'max':10, 'palette':cm.palettes.Blues}

## create and display an interactive map
Map = geemap.Map(basemap='CartoDB.DarkMatter')
Map.add_basemap('CartoDB.Positron', show=False)
Map.add_layer(flood.selfMask(), flood_vis, 'Historic floods')
Map.add_colorbar(flood_vis, label="Number of floods", layer_name="Historic floods", font_size=9)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

> **Task:** Add the input datasets to the map as individual layers. You can find the geemap documentation [here](https://geemap.org/).  
**@presenter:** Ask if someone wants to share their map and explain what functions they used to create it.

In [ ]:
#TODO: add layers to map

---
# Training dataset



In [ ]:
def pointQuery(fc:ee.FeatureCollection, img:ee.Image, prop:str) -> ee.FeatureCollection:
  """
  Returns pixel values at locations using a feature collection and an image.

  Parameters
  ----------
  fc: : ee.FeatureCollection
    Collection of points at which to query the image.
  img : ee.Image
    The image to query.
  prop : str
    Name of new property to hold the query results.

  Returns
  -------
  ee.FeatureCollection
    Input FeatureCollection with lookup values added as new property.
  """
  fc = img.reduceRegions(collection=fc, reducer=ee.Reducer.first())
  return fc.map(lambda feat: feat.set(prop, feat.get('first')))


def removeProperty(fc:ee.FeatureCollection, prop:str) -> ee.FeatureCollection:
  """
  Removes a property by name from a feature collection.

  Parameters
  ----------
  fc : ee.FeatureCollection
    Collection from which to remove the property.
  prop : str
    Property to remove.

  Returns
  -------
  ee.FeatureCollection
    Input collection without the removed property.
  """
  selectProperties = fc.propertyNames().filter(ee.Filter.neq('item', prop))
  return fc.select(selectProperties)


def createSample(img:ee.Image) -> ee.FeatureCollection:
  """
  Samples training dataset for a single flood image.

  Parameters
  ----------
  img : ee.Image
    Input image.

  Returns
  -------
  ee.FeatureCollection
    Sampled and enriched training data.
  """

  ## subtract permanent water bodies from flooded areas
  permanent = img.select('jrc_perm_water')
  water = img.select('flooded')
  flooded = water.subtract(permanent).gt(0)

  ## get the total and maximum precipitation over 14 days prior to the event end date
  end = img.getNumber('system:time_end')
  start = end.subtract(1209600000) # timestamp in milliseconds: 60 * 60 * 24 * 14 * 1000
  precSum = prec.filter(ee.Filter.date(start, end)).sum()
  precMax = prec.filter(ee.Filter.date(start, end)).max()

  ## sample equal number of flooded and non-flooded points
  sample = flooded.stratifiedSample(numPoints=SAMPLE_SIZE, classBand='flooded', geometries=True)

  ## add image id in case we want to join the event metadata later
  sample = sample.map(lambda x: x.set('eventId', img.get('system:index')))

  ## enrich sample by running point lookups on multiple datasets
  sample = pointQuery(sample, dem.mosaic(), 'demElevationAbs')
  sample = pointQuery(sample, ee.Terrain.aspect(dem.mosaic()), 'demAspect')
  sample = pointQuery(sample, ee.Terrain.slope(dem.mosaic()), 'demSlope')
  sample = pointQuery(sample, landcover.first(), 'landcover')
  sample = pointQuery(sample, hydro.select('upa'), 'upa')
  sample = pointQuery(sample, runoffPotential, 'runoffPot')
  sample = pointQuery(sample, precSum, 'precSum')
  sample = pointQuery(sample, precMax, 'precMax')

  ## remove first-property
  sample = sample.map(lambda feat: removeProperty(feat, 'first'))

  ## normalize elevation
  elevationRange = dem.mosaic().reduceRegion(geometry=img.geometry(), reducer=ee.Reducer.minMax())
  min = ee.Number(elevationRange.get('DEM_min'))
  max = ee.Number(elevationRange.get('DEM_max'))
  def normalizeElevation(feat:ee.Feature) -> ee.Feature:
    return feat.set('demElevationNorm', (ee.Number(feat.get('demElevationAbs')).subtract(min)).divide(max.subtract(min)))
  sample = sample.filter(ee.Filter.notNull(ee.List(['demElevationAbs']))).map(normalizeElevation)

  return sample

> **Task:** Try to understand the code in the cell above. Why are we using *ee.Image.stratifiedSample* in line 70 instead of using the much faster *ee.Image.Sample* method? Why are we using the complicated GEE methods in lines 65-76 and 90-94 instead of simply using plain Python? What does *ee.FeatureCollection.map* do? Why do we not just write a simple for-loop instead? Why do we normalize the elevation values?  
**@presenter:** Ask where people looked up the GEE functions. Show the documentation section in the [GEE Code editor](https://code.earthengine.google.com/) if not mentioned.  
**@presenter:**
- *stratifiedSample* samples the same number of locations within each class. Since our flood footprints mostly contain non-flooded pixels, using *stratifiedSample* is a convenient way to avoid oversampling non-flooded pixels. However, there might also be a risk of *stratifiedSample* oversampling certain flooded areas if the flood footprint is very small.
- We want to execute all the code in GEE itself so we can benefit from its optimization and parallelisation. If we would use plain Python, we would need to fetch the info from GEE server into our Colab Runtime which would make the whole process extremely inefficient.
- *map* iterates over a feature collection and applies an algorithm to each feature. This process is run in parallel on the GEE server. Using a Python for-loop instead would loose the parallel execution and also create all the problems mentioned in the last question.
- We use a global flood dataset, but look at the individual events independently. We do not want to create a bias towards the general elevation of the region where an event took place and are therefore normalizing the elevation within each image (and therefore for each event).

In [ ]:
## create enriched sample and store the result as an asset
properties = ['demAspect', 'demElevationAbs', 'demElevationNorm', 'demSlope', 'eventId', 'flooded', 'landcover', 'precMax', 'precSum', 'runoffPot', 'upa', '.geo']
sample = globalFlood.map(createSample).flatten()
sample = sample.filter(ee.Filter.notNull(properties)).distinct(properties)
task = ee.batch.Export.table.toAsset(sample, description='flood475-sampling', assetId=f"projects/{PROJECT_ID}/assets/flood475_sample")

In [ ]:
## run and monitor the task
task.start()
while task.active():
  ts = task.status()
  if ts['start_timestamp_ms']>0:
    s = round((ts['update_timestamp_ms']-ts['start_timestamp_ms'])/1000)
  else:
    s = round((ts['update_timestamp_ms']-ts['creation_timestamp_ms'])/1000)
  print(f"task '{ts['description']}' is {ts['state']} for {s} seconds")
  time.sleep(60)
task.status()

> **Task:** There are multiple places (outside of Google Colab) where we can also monitor our tasks. Can you find them?  
**@presenter:** Show the 3 places:
- [GEE Code editor](https://code.earthengine.google.com/)
- [Task Manager](https://code.earthengine.google.com/tasks)
- [Tasks Page in the Cloud Console](https://console.cloud.google.com/earth-engine/tasks?project=ee-timwaldburger-flood475)

> **Task:** Add the sampled data points to your map from above.

In [ ]:
#TODO: add data points to map

# Model training

In [57]:
## import the training dataset
sample = ee.FeatureCollection(f"projects/{PROJECT_ID}/assets/flood475_sample")
#TODO: add my training sample for people to use

## partition into 70% training and 30% validation samples
sample = sample.randomColumn('random', seed=SEED)
training = sample.filter(ee.Filter.lt('random', 0.7))
validation = sample.filter(ee.Filter.gte('random', 0.7))

## train a random forest
properties = ['demAspect', 'demElevationNorm', 'demSlope', 'landcover', 'precMax', 'precSum', 'runoffPot', 'upa']
randomForest = ee.Classifier.smileRandomForest(10)
classifier = randomForest.train(
    features=training,
    classProperty='flooded',
    inputProperties=properties
)

In [10]:
# Accuray on Training Set
train_accuracy = classifier.confusionMatrix().accuracy()
display(train_accuracy)

# Accuray on Test Set
test_accuracy = validation.classify(classifier).errorMatrix('flooded', 'classification').accuracy()
display(test_accuracy)

# Prediction

In [137]:
## get region of interest from map bounds
roi = ee.Geometry.BBox(*Map.getBounds())

## normalize elevation
elevationRange = dem.mosaic().reduceRegion(geometry=roi, reducer=ee.Reducer.minMax(), scale=30, bestEffort=True)
min = ee.Number(elevationRange.get('DEM_min'))
max = ee.Number(elevationRange.get('DEM_max'))

## calculate start and end data for precipitation data aggregation
end = ee.Date('2024-10-31T00:00:00').millis()
start = end.subtract(1209600000) # timestamp in milliseconds: 60 * 60 * 24 * 14 * 1000

## create composite of all relevant datasets
composite = ee.Image.cat(
    ee.Terrain.aspect(dem.mosaic()),
    dem.mosaic().unitScale(min, max),
    ee.Terrain.slope(dem.mosaic()),
    landcover.first(),
    prec.filter(ee.Filter.date(start, end)).max(),
    prec.filter(ee.Filter.date(start, end)).sum(),
    runoffPotential,
    hydro.select('upa')
).rename(properties)

## classify the composite
classified = composite.classify(classifier).clip(roi)

## update the map
Map.addLayer(classified, {'min':0, 'max':1}, 'Classification')
Map

Map(bottom=24297.0, center=[43.337164854911094, 3.8616943359375004], controls=(WidgetControl(options=['positio…